# Limpieza de datos

In [1]:
import pandas as pd
df = pd.read_csv('DelayedFlights.csv', low_memory=False)

## Redefinición de nombres de columnas
Como la base de datos tiene nombres de columnas con carácteres especiales, se realiza una estandarización de los nombres de columnas, reemplazando espacios, tildes y otros carácteres especiales que se puedan encontrar.
Esto se hace para evitar problemas futuros en el llamado de las columnas y para facilitar el trabajo del dataset.

In [2]:
# column "Unnamed: 0" = "id"
df['id'] = df['Unnamed: 0']
df.drop(['Unnamed: 0'], axis=1, inplace=True)
print(df.columns)

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'id'],
      dtype='object')


## Valores duplicados
Como se pudo observar en la fase de análisis exploratorio, en el dataset no existen valores duplicados, por lo que no es necesario realizar una limpieza de datos en este aspecto.

## Valores nulos
Como primera medida, se considera como regla dejar fuera las columnas que tienen más de un 50% de datos nulos debido a la falta de información suficiente. 

Como las columnas del dataset que tienen valores nulos son menores del 50%, se mantendrán y se procederá a imputarlos



In [3]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values = missing_values / df.shape[0] * 100
cols_with_more_than50_mv = missing_values[missing_values > 50]
print(missing_values)

LateAircraftDelay    35.588855
WeatherDelay         35.588855
NASDelay             35.588855
SecurityDelay        35.588855
CarrierDelay         35.588855
ActualElapsedTime     0.433043
ArrDelay              0.433043
AirTime               0.433043
ArrTime               0.367108
TaxiIn                0.367108
TaxiOut               0.023493
CRSElapsedTime        0.010223
TailNum               0.000258
dtype: float64


In [40]:
# IMPUTACIÓN DE VALORES FALTANTES

In [41]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values = missing_values / df.shape[0] * 100
print(missing_values)

Series([], dtype: float64)


# Creación de nuevas variables



__Cabe destacar que más variables serán creadas en el paso de Visualización de datos conforme sea necesario respecto a las agrupaciones y transformaciones.__